In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import os

In [ ]:
C0 = 1.66e-9
os.mkdir('graphs', exist_ok=True)  # create folder for storing graphs

def func(x, m, c):  # for fitting curve
    return m*x + c
def p(number, decimal_places=2): # format float to scientific notation (latex style)
    formatted_string = "{:.{}e}".format(number, decimal_places)
    coefficient, exponent = formatted_string.split('e')
    coefficient = "{:.{}f}".format(float(coefficient), decimal_places)
    return "{}\\times10^{{{}}}".format(coefficient, int(exponent))

In [ ]:
def do(freq, component): # open files, read data and do the calculation
    cap = pd.read_csv(f"data/{component}_csv/cap_{freq}.csv", names=["v", "x", "y"], header=None, skiprows=1)
    v = -cap["v"].to_numpy()
    vx0 = cap["x"].to_numpy()
    vy0 = cap["y"].to_numpy()

    device = pd.read_csv(f"data/{component}_csv/{component}_{freq}.csv", names=["v", "x", "y"], header=None, skiprows=1)
    vx = device["x"].to_numpy()
    vy = device["y"].to_numpy()

    C = C0 * (vx*vx0+vy*vy0)/(vx0**2+vy0**2)
    y = 1/(C**2)

    return v, y

In [ ]:
def plot_with_error(
    xs, ys, func,
    title=None,
    xlabel=None,
    ylabel=None,
    save_path = None,
    x_scale=1,
    y_scale=1e16,
    verbose = True,
    show_error = False,
    a = 0.01  # how much more to show on the x-axis beyond the data
):
    popt, pcov = curve_fit(func, xs, ys/y_scale)
    m = popt[0] * (y_scale/x_scale)
    c = popt[1] * (y_scale/x_scale)
    m_err = pcov[0, 0]**0.5 * (y_scale/x_scale)
    c_err = pcov[1, 1]**0.5 * (y_scale/x_scale)
    m_err_p = m_err*100/m
    c_err_p = c_err*100/c

    # RESULT OUPUT
    text = "If the Equation is y = mx + c\nFor the best fit curve:\n"
    text += f"    m = {m:.2e} ± {m_err:.2e} ({m_err_p:.1f}%)\n"
    text += f"    c = {c:.2e} ± {c_err:.2e} ({c_err_p:.1f}%)\n"
    if verbose:
        print(text)
        print("The curve looks like:")

    factor = (max(xs) - min(xs))*a

    xFit = np.arange(min(xs)-factor, max(xs)+factor, 1e-5) # np.arrange(start, stop, step), makes an array of numbers (range is chosen to match x data)
    
    # Plotting all the things... data points, fit, upper limit of error, lower limit of error
    plt.plot(xs, ys, marker="o", linestyle="", color="r", label="Data Points")
    plt.plot(xFit, func(xFit, m, c), label="Least Square Fit", color="k")
    if show_error:
        plt.plot(xFit, func(xFit, m+m_err, c+c_err), "b", linewidth=0.5, label="Error Boundaries")
        plt.plot(xFit, func(xFit, m-m_err, c-c_err), "b", linewidth=0.5)
    
    if title is not None:plt.title(title)
    if xlabel is not None:plt.xlabel(xlabel)
    if ylabel is not None:plt.ylabel(ylabel)
    plt.legend()
    plt.text(0.41, 0.11, text, fontsize=12, transform=plt.gcf().transFigure)
    
    if save_path is not None:plt.savefig(save_path)
    
    return m, m_err, c, c_err

In [ ]:
freqs = [500, 750, 1000, 1250, 1500]
comp = "sd"
    
e = 1.6e-19
epsilon = 11.7
epsilon0 = 8.85e-12
A = 5.38e-6

for freq in freqs:
    v, y = do(freq, comp)
    m, m_err, c, c_err = plot_with_error(
        v, y, func,
        show_error=True,
        # title=f"{freq} Hz Schottkey Diode",
        xlabel="- Reverse Bias (V)",
        ylabel="$1/C^2 (F^{-2})$",
        save_path=f"graphs/{comp}_{freq}.pdf",
        verbose=False,
        a=0.1
    )
    
    vbi = c/m
    vbi_err = vbi * (c_err/c + m_err/m)
    rho = 2/(e*epsilon*epsilon0*A**2*m)
    rho_err = rho * (m_err/m)
    
    text = f"""
        \\subsubsection{{{freq}Hz}}
            \\begin{{figure}}
                \centering
                \includegraphics[width=0.9\linewidth]{{images/{comp}_{freq}}}
                \caption{{Schottky diode graph ({freq} Hz)}}
                \label{{fig:{comp}_{freq}}}
            \end{{figure}}

            In Figure \\ref{{fig:{comp}_{freq}}}, the slope (m) = ${p(m)} \pm {p(m_err)} ({m_err*100/m:.1f}\%)$ and the intercept, (c) = ${p(c)} \pm {p(c_err)} ({c_err*100/c:.1f}\%)$. Following Eq \\ref{{eq:11}} and \\ref{{eq:10}}, the builtin voltage and doping density are:

            $$V_{{bi}} = \\frac{{{p(m)} \pm {p(m_err)}}}{{{p(c)} \pm {p(c_err)}}} = {vbi:.2f} \pm {vbi_err:.2f} V$$

            \\begin{{equation*}}
                \\begin{{split}}
                \\rho & = \\frac{{2}}{{e\\epsilon\\epsilon_0A^2m}} \\\\
                      & = {p(rho)} \pm {p(rho_err)} Dopants/m^3
                \end{{split}}
            \end{{equation*}}
"""
        
    print(text)

In [ ]:
freqs = [500, 1000, 1500]
comp = "sol"

e = 1.6e-19
epsilon = 11.7
epsilon0 = 8.85e-12
A = 3.7e-4

vbis = []
vbi_errs = []
rhos = []
rho_errs = []

for freq in freqs:
    v, y = do(freq, comp)
    m, m_err, c, c_err = plot_with_error(
        v, y, func,
        show_error=True,
        # title=f"{freq} Hz Solar Cell",
        xlabel="- Reverse Bias (V)",
        ylabel="$1/C^2 (F^{-2})$",
        save_path=f"graphs/{comp}_{freq}.pdf",
        verbose=False,
        a = 0.1
    )
    
    vbi = c/m
    vbi_err = vbi * (c_err/c + m_err/m)
    rho = 2/(e*epsilon*epsilon0*A**2*m)
    rho_err = rho * (m_err/m)
    vbis.append(vbi)
    vbi_errs.append(vbi_err)
    rhos.append(rho)
    rho_errs.append(rho_err)
    
    text = f"""
        \\subsubsection{{{freq}Hz}}
            \\begin{{figure}}
                \centering
                \includegraphics[width=0.9\linewidth]{{images/{comp}_{freq}}}
                \caption{{Solar Panel graph ({freq} Hz)}}
                \label{{fig:{comp}_{freq}}}
            \end{{figure}}

            In Figure \\ref{{fig:{comp}_{freq}}}, the slope (m) = ${p(m)} \pm {p(m_err)} ({m_err*100/m:.1f}\%)$ and the intercept, (c) = ${p(c)} \pm {p(c_err)} ({c_err*100/c:.1f}\%)$. Following Eq \\ref{{eq:11}} and \\ref{{eq:10}}, the builtin voltage and doping density are:

            $$V_{{bi}} = \\frac{{{p(m)} \pm {p(m_err)}}}{{{p(c)} \pm {p(c_err)}}} = {vbi:.2f} \pm {vbi_err:.2f} V$$

            \\begin{{equation*}}
                \\begin{{split}}
                \\rho & = \\frac{{2}}{{e\\epsilon\\epsilon_0A^2m}} \\\\
                      & = {p(rho)} \pm {p(rho_err)} Dopants/m^3
                \end{{split}}
            \end{{equation*}}
"""
        
    print(text)

print()